## NN from scratch
inspired by https://towardsdatascience.com/math-neural-network-from-scratch-in-python-d6da9f29ce65

In [ ]:
import numpy as np

# Base class
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError

    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

### Forward propagation
![](https://miro.medium.com/v2/resize:fit:1252/format:webp/0*3cUc4jsQlHsoovn9.png)

### Backward propagation

![](https://miro.medium.com/v2/resize:fit:1400/format:webp/0*bL0jVGoQiH_TsrvT.png)

- Remember that $E$ is a **scalar** and $X$, $Y$ are **matrices**.
    - $ \displaystyle \frac{\partial E}{\partial X} = \left[\begin{matrix} \frac{\partial E}{\partial x_1} \\ \cdots \\ \frac{\partial E}{\partial x_N} \end{matrix}\right], \frac{\partial E}{\partial Y} = \left[\begin{matrix} \frac{\partial E}{\partial y_1} \\ \cdots \\ \frac{\partial E}{\partial y_N} \end{matrix}\right]. $

- Why $\frac{\partial E}{\partial X}$?
    - Don’t forget, the output of one layer is the input of the next layer. Which means $\frac{\partial E}{\partial X}$ for one layer is $\frac{\partial E}{\partial Y}$ for the previous layer ! 

### Weight initialization
- 초기값을 모두 0으로 설명한다면 역전파 과정에서 가중치들의 update가 동일하게 이루어지기 때문에 deep neural network의 이점을 누리기 힘들 수 있음.
- 정규분포의 형태로 초기화한 경우, 표준편차가 크다면 sigmoid 함수인 경우 기울기 소실 문제가 발생할 수 있음.
- 아래 초기화 방법들의 선택 기준은 명확하지 않고 여러 방법을 테스트하는 것이 필요함.

1. LeCun initialization (https://cseweb.ucsd.edu/classes/wi08/cse253/Handouts/lecun-98b.pdf)
    - Normal initialization
        - $\displaystyle W \sim N(0, Var(W)), Var(W) = \sqrt{\frac{1}{n_{in}}}$
    - Uniform initialization
        - $\displaystyle W \sim U(-\sqrt{\frac{1}{n_{in}}}, \sqrt{\frac{1}{n_{in}}})$
2. Xavier(Glorot) initialization (https://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)
    - Normal initialization
        - $\displaystyle W \sim N(0, Var(W)), Var(W) = \sqrt{\frac{2}{n_{in}+n_{out}}}$
    - Uniform initialization
        - $\displaystyle W \sim U(-\sqrt{\frac{6}{n_{in}+n_{out}}}, \sqrt{\frac{6}{n_{in}+n_{out}}})$
    - sigmoid, tanh에서 효율적이지만 relu의 경우 비효율적
3. He initialization (https://arxiv.org/abs/1502.01852)
    - Normal initialization
        - $\displaystyle W \sim N(0, Var(W)), Var(W) = \sqrt{\frac{2}{n_{in}}}$
    - Uniform initialization
        - $\displaystyle W \sim U(-\sqrt{\frac{6}{n_{in}}}, \sqrt{\frac{6}{n_{in}}})$
    - relu의 경우 주로 사용할 수 있음.


### Bias initialization
- Initializing to zero is general.

In [ ]:
# inherit from base class Layer
class FCLayer(Layer):
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size):
        # He normal initialization
        self.weights = np.random.randn(output_size, input_size) * np.sqrt(2 / input_size)
        # self.weights = np.random.rand(output_size, input_size) - 0.5
        self.bias = np.zeros((output_size, 1))
        # self.bias = np.random.rand(output_size, 1) - 0.5

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.weights, self.input) + self.bias
        return self.output

    # computes dL/dW, dL/dB for a given output_error=dL/dY. Returns input_error=dL/dX.
    def backward_propagation(self, output_error, learning_rate):
        weights_error = np.outer(output_error, self.input)  # dL/dW = dL/dY * dY/dW
        input_error = np.dot(self.weights.T, output_error)  # dL/dX = dL/dY * dY/dX
        bias_error = output_error                       # dL/dB = dL/dY * dY/dB

        # update parameters
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * bias_error
        return input_error

### Activation function layer

![](https://miro.medium.com/v2/resize:fit:640/format:webp/1*xl7UacJfCUAk_KqX3WI49Q.png)

In [ ]:
# inherit from base class Layer
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    # returns the activated input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    # Returns input_error=dE/dX for a given output_error=dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

In [ ]:
# Activation functions and their derivatives
def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    return 1-np.tanh(x)**2

def relu(x):
    return np.maximum(0,x)

def relu_derivative(x):
    x[x<=0] = 0
    x[x>0] = 1
    return x

def leaky_relu(x):
    return np.maximum(0.01*x,x)

def leaky_relu_derivative(x):
    x[x<=0] = 0.01
    x[x>0] = 1
    return x

def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x)*(1-sigmoid(x))

def swish(x):
    return x*sigmoid(x)

def swish_derivative(x):
    return sigmoid(x) + x*sigmoid_derivative(x)

def softmax(x):
    x = np.clip(x, -100, 100)   # overflow 방지
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def softmax_derivative(x):
    s = softmax(x)
    return s * (1 - s)

# Loss functions and their derivatives
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2))

def mse_derivative(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size

def cross_entropy(y_true, y_pred):
    return -np.log(y_pred[y_true==1]).mean()

def cross_entropy_derivative(y_true, y_pred):
    return y_pred-y_true

In [ ]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime
    
    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j].reshape(10, 1), output)

                # backward propagation
                error = self.loss_prime(y_train[j].reshape(10, 1), output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            # calculate average error on all samples
            err /= samples
            # # print error 100 times
            # if i % (epochs/100) == 0:
            print(f'epoch {i}/{epochs}   error={err}')

    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []
        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i].reshape(28*28, -1)
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)
        return result
    
    def get_accuracy(self, x_test, y_test):
        y_pred = self.predict(x_test)
        y_pred = np.argmax(y_pred, axis=1)
        y_test = np.argmax(y_test, axis=1)
        return np.sum(y_pred == y_test) / len(y_test)

In [ ]:
from keras.datasets import mnist
from keras.utils import np_utils

# load MNIST from server
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# training data : 60000 samples
# reshape and normalize input data
x_train = x_train.reshape(x_train.shape[0], 28*28, 1)
x_train = x_train.astype('float32')
x_train /= 255
# encode output which is a number in range [0,9] into a vector of size 10
# e.g. number 3 will become [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
y_train = np_utils.to_categorical(y_train).reshape(y_train.shape[0], 10, 1)

# same for test data : 10000 samples
x_test = x_test.reshape(x_test.shape[0], 28*28, 1)
x_test = x_test.astype('float32')
x_test /= 255
y_test = np_utils.to_categorical(y_test).reshape(y_test.shape[0], 10, 1)

In [ ]:
# Network
net = Network()
net.add(FCLayer(28*28, 100))                # input_shape=(28*28, 1)    ;   output_shape=(100, 1)
net.add(ActivationLayer(relu, relu_derivative))
# net.add(FCLayer(100, 50))                   # input_shape=(100, 1)      ;   output_shape=(50, 1)
# net.add(ActivationLayer(tanh, tanh_derivative))
net.add(FCLayer(100, 10))                    # input_shape=(50, 1)       ;   output_shape=(10, 1)
net.add(ActivationLayer(softmax, softmax_derivative))

# train on 1000 samples
# as we didn't implemented mini-batch GD, training will be pretty slow if we update at each iteration on 60000 samples...
net.use(cross_entropy, cross_entropy_derivative)
net.fit(x_train[:1000], y_train[:1000], epochs=20, learning_rate=0.01)

In [ ]:
print(f'Accuracy for training samples: {net.get_accuracy(x_train[:1000], y_train[:1000])}')
print(f'Accuracy for test samples: {net.get_accuracy(x_test[:200], y_test[:200])}')